-------------------------------------------------------------------------------
### Text Translation (Sync) using SAP Translation Hub via LLM
-------------------------------------------------------------------------------


---------------------------------------------------------------------------
What this script does
1) Loads OAuth credentials and API base URL from a .env file.
2) Fetches an access token via client-credentials.
3) Calls the **Document Translation** sync endpoint with a JSON body.
   - Here we send plain text with contentType=text/plain.
4) Prints the raw JSON response (so you can see exactly what came back).
--------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------
Environment variables expected in .env
- AUTH_URL          : OAuth token URL from your service key (…/oauth/token)
- CLIENT_ID         : clientid from service key
- CLIENT_SECRET     : clientsecret from service key
- DOCTRANS_BASE     : e.g. https://document-translation.api.<region>.translationhub.cloud.sap
-------------------------------------------------------------------------------------------------


Prerequisites

- Python 3.10+

- Libraries given below
    - pip install python-dotenv requests python-docx

- .env file in the working directory with required values
- input file or text you want to translate

In [1]:
import os, time, requests
from dotenv import load_dotenv

# loads variables from .env in current directory
load_dotenv()

True

In [2]:
# Checks if all the environment prequesties are met
def require_env(name: str) -> str:
    v = os.getenv(name)
    if not v:
        raise RuntimeError(f"Missing env: {name}")
    return v


# access from .env in current directory
auth_url     = require_env("AUTH_URL")
client_id     = require_env("CLIENT_ID")
client_secret = require_env("CLIENT_SECRET")
translation_base_url = require_env("DOCTRANS_BASE")

In [3]:
# code  to fetch the access token
def get_token():
    # Prepare the payload and headers
    payload = {
        "grant_type": "client_credentials"
    }
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    # Make the POST request to obtain the token
    response = requests.post(auth_url, data=payload, headers=headers, auth=(client_id, client_secret))

    # Check if the request was successful
    if response.status_code == 200:
        access_token = response.json().get("access_token")
        print("Access token obtained successfully.")
    else:
        print(f"Failed to obtain access token: {response.status_code} - {response.text}")
    
    return access_token

In [4]:
# getting the acess token by calling the get token function
access_token=get_token()

Access token obtained successfully.


##### The code is responsible for conversion of text from one language to other using LLM
Required paramaters
- sourceLanguage(optional) - The original source language of given text
- targetLanguage - The target conversion language for the given text
- model- "LLM" in case of using the LLM
- encoding
    - plain - If giving plain language text
    - base64 - if objects are convered to 
- Content-Type
    - text/plain
    - text/HTML
    - multipart
- data : The text which you want to convert

In [5]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

body = {
   "sourceLanguage": "de-DE",
   "targetLanguage": "en-US",
   "encoding": "plain",
   "model": "llm",
   "data": "Dieser Satz soll übersetzt werden"
}
r = requests.post(f"{translation_base_url}/api/v1/translation", headers=headers, json=body, timeout=60)
r.raise_for_status()
print(r.json())



{'contentType': 'text/html', 'encoding': 'plain', 'sourceLanguage': 'de-DE', 'targetLanguage': 'en-US', 'model': 'llm', 'data': 'This sentence should be translated'}


The r.json() gives the json object as shown below

```json
{
    'contentType': 'text/plain',
    'encoding': 'plain', 
    'sourceLanguage': 'de-DE',
    'targetLanguage': 'en-US',
    'model': 'llm',
    'data': 'This sentence is to be translated.'
}

Which contains the desired translated text in "data" key

In [6]:
print(r.json().get("data"))

This sentence should be translated


### Summary

This notebook demonstrates that you can convert text from to english using SAP BTP Document Translation Service.